In [ ]:
import json
import os
import pandas
from pprint import pprint

In [ ]:
dirs = ['biorxiv_medrxiv']

for d in dirs:
    files = os.listdir(d)
    for file in files:
        filepath = os.path.join(d, file)
        with open(filepath, 'rb') as f:
            file_data = json.load(f)
            title = file_data['metadata']['title']
            abstract = file_data['abstract']  # List of paragraphs in abstract
            body_text = file_data['body_text']
            # Note: Sometimes abstract is empty!
            if len(abstract) == 0 or len(body_text) == 0:
                print(len(abstract))  # Num. paragraphs in abstract
                # print(len(body_text))  # Num. paragraphs in body